In [24]:
import sys
sys.path.append('..')


In [25]:
import numpy as np
from math import log2
from scripts.utils import load_data

In [26]:
# Import binary data
data_1 = load_data('../data/data_1.txt')
data_2 = load_data('../data/data_2.txt')
print(data_1)
print(data_2)
print(type(data_1))

(array([[0., 1., 0., 1.],
       [1., 1., 1., 1.],
       [0., 0., 0., 1.]]), array([1, 1, 0]))
(array([[0., 1., 0., 1.],
       [1., 1., 1., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]]), array([1, 1, 0, 0]))
<class 'tuple'>


In [27]:
def calculate_entropy(Y):
    classes, counts = np.unique(Y, return_counts=True)
    p_C = counts / counts.sum()
    calculate_entropy = np.sum([-p_c * log2(p_c) for p_c in p_C])
    return calculate_entropy

print(data_1[1])
print(calculate_entropy(data_1[1]))


[1 1 0]
0.9182958340544896


In [28]:
def calculate_information_gain(feature, labels):
    H_labels = calculate_entropy(labels)
    classes, counts = np.unique(feature, return_counts=True)

    H_feature = 0
    for cls, count in zip(classes, counts):
        cls_labels = labels[feature == cls]
        p_t = count / len(labels)
        H_t = calculate_entropy(cls_labels)
        H_feature += p_t * H_t

    ig = H_labels - H_feature
    return ig

print(calculate_information_gain(data_1[0].T[0], data_1[1]))

0.2516291673878229


In [ ]:
def DT_train_binary(X: np.ndarray, Y):
    # Calculate the Entropy (H) for the entire training set
    entropy = calculate_entropy(Y)
    print(entropy)

    igs = []
    # Split training data using each feature
    for idx, col in enumerate(X.T):
        
        # Calculate Information Gain for each split

        igs.append(calculate_information_gain(col, Y))
        print(idx)
        print(calculate_information_gain(col, Y))
        pass
        

    # Choose to split on the feature that gives the best IG
    

    pass

temp = data_1
print(temp[0])
print(temp[1])
DT_train_binary(temp[0], temp[1])

[[0. 1. 0. 1.]
 [1. 1. 1. 1.]
 [0. 0. 0. 1.]]
[1 1 0]
0.9182958340544896
0
0.2516291673878229
1
0.9182958340544896
2
0.2516291673878229
3
0.0
